# Overlay land cover and land usage

In [3]:
import numpy as np
import pandas as pd
import time
import math
from skimage import io
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Polygon, Point
import folium
import contextily as cx
import osmnx as ox

C:\Users\bahr\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\bahr\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [5]:
# set up dirs

dir_h = "YOUR_DIRECTORY/"
dir_rs = "YOUR_DIRECTORY/"

## Load household data set

In [ ]:
shp21 = pd.read_pickle(dir_h + "shp21_preliminar.pkl")
shp21 = shp21[["idhous21", "g_lat", "g_lon", "npa"]].drop_duplicates().reset_index(drop=True)

## Scrape land usage from OpenStreetMap

In [ ]:
def land_use_extraction(inputs, tags):
    features = ox.features_from_point(inputs[1:], tags=tags, dist=630)
    features = features.reset_index()
    columns = ["geometry"]
    for i in ["idhous21", "osmid", "element_type", "amenity", "shop", "landuse", "leisure", "building"]:
         if i in features.columns:
            columns.append(i)
    features = features[(features.geometry.type.isin(["Polygon", "Point"])) & (features.element_type.isin(["way", "node"]))][columns]
    features["idhous21"] = int(inputs[0])
    return features

def overlapping_polygons(data):
    df = data[data.element_type=="way"]
    df = df[df.landuse != "residential"] # exclude landuse residenial
    df = df[~df.duplicated(subset="geometry", keep="first")]
    parent_idx, children_idx = df.sindex.query(df.geometry, predicate="contains")
    mask = parent_idx != children_idx
    parent_idx = parent_idx[mask]
    children_idx = children_idx[mask]

    if len(parent_idx) == 0:
        data = data.reset_index(drop=True)
    else:
        for parent in np.unique(parent_idx):
            parent_mask = parent_idx == parent
            child_idx = children_idx[parent_mask]
            parent_poly = df.iloc[[parent]]
            for child in child_idx:
                if len(parent_poly) == 0:
                    parent_poly = df_land_use.iloc[[parent]]
                parent_poly = gpd.overlay(parent_poly, df.iloc[[child]], how="difference", keep_geom_type=True)
            df = pd.concat([df, parent_poly]) # add new parent
        df = df.drop(df.index[np.unique(parent_idx)]) # drop deprecated parent polygon
        #df = df.reset_index(drop=True)
        data = pd.concat([df, data[data.element_type=="node"]])
        data = data.reset_index(drop=True)
    return data

def post_processing(data):
    # all considered amenities and shop categories
    amenities = ["bar", "biergarten", "cafe", "fast_food", "food_court", "ice_cream", "pub", "restaurant",
                "college", "kindergarten", "library", "toy_library", "training", "music_school", "school",
                "university", "car_wash", "fuel", "atm", "bank", "clinic", "dentist", "doctors",
                "hospital", "nursing_home", "pharmacy", "social_facility", "veterinary", "arts_centre",
                "casino", "cinema", "community_centre", "events_venue", "music_venue", "nightclub",
                "social_centre", "theatre", "post_office", "townhall", "parcel_locker", "recycling",
                "marketplace", "alcohol", "bakery", "beverages", "butcher", "cheese", "coffee",
                "confectionery", "convenience", "deli", "dairy", "farm", "frozen_food", "greengrocer",
                "health_food", "pasta", "pastry", "seafood", "spices", "tea", "wine",
                "food", "departement_store", "general", "kiosk", "mall", "supermarket",
                "boutique", "clothes", "fabric", "shoes", "tailor", "watches", "wool", "charity",
                "second_hand", "variety_store", "beauty", "chemist", "cosmetics", "hairdresser",
                 "optician", "perfumery", "appliance", "doityourself", "electrical", "florist",
                "garden_center", "hardware", "houseware", "paint", "furniture", "electronics",
                "mobile_phone", "bicycle", "car", "outdoor", "sports", "art", "books", "stationery",
                "dry_cleaning", "toys"]

    # categorize amenities
    amenities_map = {'bar': 'CCE',
                     'biergarten': 'CCE',
                     'cafe': 'CCE',
                     'fast_food': 'CCE',
                     'food_court': 'CCE',
                     'ice_cream': 'CCE',
                     'pub': 'CCE',
                     'restaurant': 'CCE',
                     'college': 'public_services',
                     'kindergarten': 'public_services',
                     'library': 'public_services',
                     'toy_library': 'public_services',
                     'training': 'public_services',
                     'music_school': 'public_services',
                     'school': 'public_services',
                     'university': 'public_services',
                     'car_wash': 'commercial',
                     'fuel': 'commercial',
                     'atm': 'public_services',
                     'bank': 'public_services',
                     'clinic': 'public_services',
                     'dentist': 'public_services',
                     'doctors': 'public_services',
                     'hospital': 'public_services',
                     'nursing_home': 'public_services',
                     'pharmacy': 'public_services',
                     'social_facility': 'public_services',
                     'veterinary': 'public_services',
                     'arts_centre': 'CCE',
                     'casino': 'CCE',
                     'cinema': 'CCE',
                     'community_centre': 'public_services',
                     'events_venue': 'CCE',
                     'music_venue': 'CCE',
                     'nightclub': 'CCE',
                     'social_centre': 'public_services',
                     'theatre': 'CCE',
                     'post_office': 'public_services',
                     'townhall': 'public_services',
                     'parcel_locker': 'public_services',
                     'recycling': 'public_services',
                     'marketplace': 'groceries',
                     'alcohol': 'groceries',
                     'bakery': 'groceries',
                     'beverages': 'groceries',
                     'butcher': 'groceries',
                     'cheese': 'groceries',
                     'coffee': 'groceries',
                     'confectionery': 'groceries',
                     'convenience': 'groceries',
                     'deli': 'groceries',
                     'dairy': 'groceries',
                     'farm': 'groceries',
                     'frozen_food': 'groceries',
                     'greengrocer': 'groceries',
                     'health_food': 'groceries',
                     'pasta': 'groceries',
                     'pastry': 'groceries',
                     'seafood': 'groceries',
                     'spices': 'groceries',
                     'tea': 'groceries',
                     'wine': 'groceries',
                     'food': 'groceries',
                     'departement_store': 'commercial',
                     'general': 'groceries',
                     'kiosk': 'commercial',
                     'mall': 'commercial',
                     'supermarket': 'groceries',
                     'boutique': 'commercial',
                     'clothes': 'commercial',
                     'fabric': 'commercial',
                     'shoes': 'commercial',
                     'tailor': 'commercial',
                     'watches': 'commercial',
                     'wool': 'commercial',
                     'charity': 'commercial',
                     'second_hand': 'commercial',
                     'variety_store': 'commercial',
                     'beauty': 'commercial',
                     'chemist': 'commercial',
                     'cosmetics': 'commercial',
                     'hairdresser': 'commercial',
                     'optician': 'commercial',
                     'perfumery': 'commercial',
                     'appliance': 'commercial',
                     'doityourself': 'commercial',
                     'electrical': 'commercial',
                     'florist': 'commercial',
                     'garden_center': 'commercial',
                     'hardware': 'commercial',
                     'houseware': 'commercial',
                     'paint': 'commercial',
                     'furniture': 'commercial',
                     'electronics': 'commercial',
                     'mobile_phone': 'commercial',
                     'bicycle': 'commercial',
                     'car': 'commercial',
                     'outdoor': 'commercial',
                     'sports': 'commercial',
                     'art': 'commercial',
                     'books': 'commercial',
                     'stationery': 'commercial',
                     'dry_cleaning': 'commercial',
                     'toys': 'commercial',
    }

    data = item_clean
    poly_columns = ["idhous21", "osmid", "amenity", "shop", "landuse", "leisure", "building", "geometry"]
    columns = []
    for i in poly_columns:
        if i in data.columns:
            columns.append(i)
    data_poly = data[data.element_type=="way"][columns].reset_index(drop=True)

    columns = []
    point_columns = ["amenity", "shop", "geometry"]
    for i in point_columns:
        if i in data.columns:
            columns.append(i)
    data_point = data[data.element_type=="node"][columns]
    data_point = gpd.overlay(data_point, data_poly, how="intersection", keep_geom_type=True)

    if "amenity_1" in data_point.columns and "shop_1" in data_point.columns:
        data_point["amenity_point"] = np.where(data_point.shop_1.isna(), data_point.amenity_1, data_point.shop_1)
    elif "amenity_1" in data_point.columns:
        data_point["amenity_point"] = data_point.amenity_1
    else:
        data_point["amenity_point"] = pd.NA
    
    if "amenity" not in data_poly.columns:
        data_poly["amenity"] = pd.NA
        
    data_point = data_point[(~data_point.amenity_point.isna()) & (data_point.amenity_point.isin(amenities))][["osmid", "amenity_point"]]
    data_point["amenity_category"] = data_point.amenity_point.map(amenities_map)
    #data_point = data_point[~data_point.duplicated(subset=["osmid", "amenity_category"], keep="first")].reset_index(drop=True) # remove amenities with same polygone ID and same amenity type
    data_poly = data_poly.merge(data_point[["osmid", "amenity_point"]], how="left", on="osmid")
    if "shop" in data_poly.columns:
        data_poly["amenity"] = np.where(data_poly.shop.isna(), data_poly.amenity, data_poly.shop)
    data_poly["building"] = np.where((~data_poly.amenity.isna()) & (data_poly.building=="yes"), data_poly.amenity, data_poly.building)
    data_poly["amenity"] = np.where(~data_poly.amenity_point.isna(), data_poly.amenity_point, data_poly.amenity)
    data_poly["amenity"] = data_poly.amenity.map(amenities_map)
    data_poly["amenity_count"] = data_poly.groupby(["osmid", "amenity"]).amenity.transform("count")
    data_poly = data_poly[~data_poly.duplicated(subset=["osmid", "amenity"], keep="first")].reset_index(drop=True)
    data_poly["amenity_weight_sum"] = data_poly.groupby("osmid").amenity_count.transform("sum")
    data_poly["amenity_weight"] = np.where(data_poly.amenity_count.isna(), 1.0,  (data_poly.amenity_count / data_poly.amenity_weight_sum))
    data_poly["osmid_count"] = data_poly.groupby("osmid").osmid.transform("count")
    return data_poly

In [ ]:
# 
# 1. case: building == amenity -> no change
# create land_usage
# 2. case: amenity_point is NAN -> land_usage (not use commercial building)
# 3. case: building != amenity & building not in apartments, residential, house, detached, terrace, dormitory, bungalow, apartments;school & count ==1 -> replace with amenity_point
# 4. case: building != amenity & building in apartments, residential, house, detached, terrace, dormitory, bungalow, apartments;school & count ==1 -> doublicate observation overlapping polygones!!!
# 5. case: count >1 keep duplicates and divide area by count value

In [ ]:
land_usage = pd.DataFrame()
img_count_ten=0
img_count=0
tags = {"landuse": True,
        "leisure": True,
       "building": True,
       "amenity": True,
       "shop": True}
    
for i in shp21.index:
    inputs = (shp21.iloc[i].idhous21, shp21.iloc[i].g_lat, shp21.iloc[i].g_lon)
    item_ext = land_use_extraction(inputs, tags)
    item_clean = overlapping_polygons(item_ext)
    item = post_processing(item_clean)
    land_usage = pd.concat([land_usage, item])
    
    img_count_ten += 1
    if img_count_ten == 10:
        img_count += 1
        img_count_ten=0
        print(img_count)

In [28]:
#land_usage.to_pickle(dir_h + "land_usage.pkl")
land_usage = pd.read_pickle(dir_h + "land_usage.pkl")

#### Dual use buildings

In [32]:
# duplicate osmid polygon if building has dual use (shop and inhabited)

land_usage["double_usage"] =  np.where(((~land_usage.amenity.isna()) & (land_usage.building.isin(["apartments", "residential", "house", "detached", "retirement_home", "terrace", "semidetached_house", "hut"]))), True, False)
land_double = land_usage[(land_usage.double_usage==True)]
land_double = land_double[~land_double.duplicated(subset=["osmid", "idhous21"], keep="first")]
land_double["amenity"] = "inhabited"
land_double["osmid_count"] = land_double.osmid_count +1
land_usage = pd.concat([land_usage, land_double]).reset_index(drop=True)
land_usage["osmid_count"] = np.where((land_usage.double_usage==True) & (land_usage.amenity!="inhabited"), land_usage.osmid_count+1, land_usage.osmid_count)

In [33]:
land_usage["amenity_count"] = np.where(land_usage.amenity=="inhabited", land_usage.amenity_weight_sum, land_usage.amenity_count)
land_usage["amenity_weight_sum"] = land_usage.groupby(["idhous21", "osmid"]).amenity_count.transform("sum")
land_usage["amenity_weight"] = np.where(land_usage.amenity_count.isna(), 1.0, (land_usage.amenity_count / land_usage.amenity_weight_sum))

#### Categorize green land usage areas

In [34]:
green_land_usage_map = {"forest": "forest",
                        "park": "parks",
                        "pavilion": "parks",
                        "pavillon": "parks",
                        "recreation_ground": "recreation",
                        "pitch": "recreation",
                        "swimming_pool": "recreation",
                        "track": "recreation",
                        "fitness_station": "recreation",
                        "dog_park": "recreation",
                        "swimming_area": "recreation",
                        "miniature_golf": "recreation",
                        "horse_riding": "recreation",
                        "Flussbad": "recreation",
                        "golf_course": "recreation",
                        "beach_resort": "recreation",
                        "table_tennis_table": "recreation", 
                        "practice_pitch": "practicrecreatione_pitch", 
                        "picnic_table": "recreation", 
                        "small_animal_park": "recreation",
                        "garden": "garden",
                        "flowerbed": "garden",
                        "playground": "playground",
                        "grass": "meadows",
                        "meadow": "meadows",
                        "village_green": "meadows",
                        "greenfield": "meadows",
                        "greenery": "meadows",
                        "shrubs": "meadows",
                        "pasture": "meadows",
                        "hedge": "meadows",
                        "fallow": "meadows",
                        "common": "meadows",
                        "allotments": "allotments",
                        "farmland": "agriculture",
                        "vineyard": "agriculture",
                        "farmyard": "agriculture",
                        "orchard": "agriculture",
                        "orchard": "agriculture",
                        "greenhouse_horticulture": "agriculture",
                        "plant_nursery": "agriculture",
                        "brownfield": "agriculture"}

In [35]:
land_usage["landuse"] = land_usage.landuse.map(green_land_usage_map).fillna(land_usage.landuse)
land_usage["leisure"] = land_usage.leisure.map(green_land_usage_map).fillna(land_usage.leisure)

In [36]:
mix_landuse_map = {'apartments': 'residential',
                   'residential': 'residential',
                   'house': 'residential',
                   'detached': 'residential',
                   'terrace': 'residential',
                   'semidetached_house': 'residential',
                   'dormitory': 'residential', 
                   'appartments': 'residential',
                   'dormitory': 'residential',
                   'inhabited': 'residential',
                   'CCE': 'CCE',
                   'bar': 'CCE',
                   'biergarten': 'CCE',
                   'cafe': 'CCE',
                   'fast_food': 'CCE',
                   'food_court': 'CCE',
                   'ice_cream': 'CCE',
                   'pub': 'CCE',
                   'restaurant': 'CCE',
                   'arts_centre': 'CCE',
                   'casino': 'CCE',
                   'cinema': 'CCE',
                   'events_venue': 'CCE',
                   'music_venue': 'CCE',
                   'nightclub': 'CCE',
                   'theatre': 'CCE',
                   'outdoor_seating': 'CCE',
                   'stadium': 'CCE',
                   'bleachers': 'CCE',
                   'grandstand': 'CCE',
                   'bandstand': 'CCE',
                   'museum': 'CCE',
                   'castle': 'CCE',
                   'recreation': 'recreation_use',
                   'sports_centre': 'recreation_use',
                   'sports_hall': 'recreation_use',
                   'ice_rink': 'recreation_use',
                   'water_park': 'recreation_use',
                   'paddling_pool': 'recreation_use',
                   'dance': 'recreation_use',
                   'fitness_centre': 'recreation_use',
                   'sauna': 'recreation_use',
                   'swimming_pool;ice_rink': 'recreation_use',
                   'riding_hall': 'recreation_use',
                   'sport_hall': 'recreation_use',
                   'bowling_alley': 'recreation_use',
                   'trampoline_park': 'recreation_use',
                   'parks': 'recreation_use',
                   'park': 'recreation_use',
                   'pavilion': 'recreation_use',
                   'pavillon': 'recreation_use',
                   'public_services': 'public_services',
                   'college': 'public_services',
                   'kindergarten': 'public_services',
                   'library': 'public_services',
                   'toy_library': 'public_services',
                   'training': 'public_services',
                   'music_school': 'public_services',
                   'school': 'public_services',
                   'university': 'public_services',
                   'atm': 'public_services',
                   'bank': 'public_services',
                   'clinic': 'public_services',
                   'dentist': 'public_services',
                   'doctors': 'public_services',
                   'hospital': 'public_services',
                   'nursing_home': 'public_services',
                   'pharmacy': 'public_services',
                   'social_facility': 'public_services',
                   'veterinary': 'public_services',
                   'community_centre': 'public_services',
                   'social_centre': 'public_services',
                   'post_office': 'public_services',
                   'townhall': 'public_services',
                   'parcel_locker': 'public_services',
                   'recycling': 'public_services',
                   'playground': 'public_services',
                   'park': 'public_services',
                   'public': 'public_services',
                   'train_station': 'public_services',
                   'railway': 'public_services',
                   'transportation': 'public_services',
                   'chapel': 'public_services',
                   'civic': 'public_services',
                   'church': 'public_services',
                   'meadow': 'public_services',
                   'college': 'public_services',
                   'fire_station': 'public_services',
                   'schoolyard': 'public_services',
                   'religious': 'public_services',
                   'cathedral': 'public_services',
                   'temple': 'public_services',
                   'synagogue': 'public_services',
                   'mosque': 'public_services',
                   'public_transport': 'public_services',
                   'hospice': 'public_services',
                   'monastery': 'public_services',
                   'public_services': 'public_services',
                   'school;kindergarten': 'public_services',
                   'place_of_worship': 'public_services',
                   'government': 'public_services',
                   'police': 'public_services',
                   'playground': 'public_services',
                   'commercial': 'commercial',
                   'car_wash': 'commercial',
                   'fuel': 'commercial',
                   'departement_store': 'commercial',
                   'general': 'groceries',
                   'kiosk': 'commercial',
                   'mall': 'commercial',
                   'interior_decoration': 'commercial',
                   'boutique': 'commercial',
                   'clothes': 'commercial',
                   'fabric': 'commercial',
                   'shoes': 'commercial',
                   'tailor': 'commercial',
                   'watches': 'commercial',
                   'wool': 'commercial',
                   'charity': 'commercial',
                   'second_hand': 'commercial',
                   'variety_store': 'commercial',
                   'beauty': 'commercial',
                   'chemist': 'commercial',
                   'cosmetics': 'commercial',
                   'hairdresser': 'commercial',
                   'optician': 'commercial',
                   'perfumery': 'commercial',
                   'appliance': 'commercial',
                   'doityourself': 'commercial',
                   'electrical': 'commercial',
                   'florist': 'commercial',
                   'garden_center': 'commercial',
                   'hardware': 'commercial',
                   'houseware': 'commercial',
                   'paint': 'commercial',
                   'furniture': 'commercial',
                   'electronics': 'commercial',
                   'mobile_phone': 'commercial',
                   'bicycle': 'commercial',
                   'car': 'commercial',
                   'outdoor': 'commercial',
                   'sports': 'commercial',
                   'art': 'commercial',
                   'books': 'commercial',
                   'stationery': 'commercial',
                   'dry_cleaning': 'commercial',
                   'retail': 'commercial',
                   'supermarket': 'groceries',
                   'toys': 'commercial',
                   'groceries': 'groceries',
                   'marketplace': 'groceries',
                   'alcohol': 'groceries',
                   'bakery': 'groceries',
                   'beverages': 'groceries',
                   'butcher': 'groceries',
                   'cheese': 'groceries',
                   'coffee': 'groceries',
                   'confectionery': 'groceries',
                   'convenience': 'groceries',
                   'deli': 'groceries',
                   'dairy': 'groceries',
                   'farm': 'groceries',
                   'frozen_food': 'groceries',
                   'greengrocer': 'groceries',
                   'health_food': 'groceries',
                   'pasta': 'groceries',
                   'pastry': 'groceries',
                   'seafood': 'groceries',
                   'spices': 'groceries',
                   'tea': 'groceries',
                   'wine': 'groceries',
                   'food': 'groceries', 
                   'industrial': 'industry',
                   'warehouse': 'industry',
                   'quarry': 'industry',
                   'storage_tank': 'industry',
                   'hangar': 'industry',
                   'container': 'industry',
                   'manufacture': 'industry',
                   'jail': 'industry',
                   'tank': 'industry',
                   
}

In [37]:
land_usage["building"] = np.where(~land_usage.amenity.isna(), land_usage.amenity, land_usage.building)
land_usage["landuse_mixed"] = land_usage.landuse.map(mix_landuse_map).fillna(land_usage.landuse)
land_usage["leisure_mixed"] = land_usage.leisure.map(mix_landuse_map).fillna(land_usage.leisure)
land_usage["building_mixed"] = land_usage.building.map(mix_landuse_map).fillna(land_usage.building)

In [38]:
land_usage["use_green"] = np.where(land_usage.leisure.isna(), land_usage.landuse, land_usage.leisure)
land_usage["use_mixed"] = np.where(land_usage.building_mixed.isna(), land_usage.leisure_mixed, land_usage.building_mixed)
land_usage["use_mixed"] = np.where(land_usage.use_mixed.isna(), land_usage.landuse_mixed, land_usage.use_mixed)
land_usage["use_mixed"] = np.where((land_usage.leisure_mixed=="recreation_use") & (land_usage.building_mixed=="public_services"), "recreation_use", land_usage.use_mixed)

# recode all remaining categories to other
land_usage["use_green"] = np.where(land_usage.use_green.isin(["forest", "parks", "recreation",
                                                             "garden", "playground", "meadows",
                                                             "allotments", "agriculture"]), land_usage.use_green, "other")
land_usage["use_mixed"] = np.where(land_usage.use_mixed.isin(["residential", "commercial", "groceries",
                                                             "CCE", "recreation_use", "public_services",
                                                             "industry"]), land_usage.use_mixed,
                                   np.where(land_usage.use_mixed=="yes", "residential", "other")) # recode building yes to residential

In [39]:
land_usage.use_mixed.value_counts()

use_mixed
residential        3960217
other               972754
public_services     176582
commercial          129162
recreation_use      117898
CCE                 112790
groceries            50325
industry             34935
Name: count, dtype: int64

In [40]:
land_usage.use_green.value_counts()

use_green
other          5157508
meadows         110285
garden           91633
recreation       91255
agriculture      30583
playground       26753
parks            21837
forest           19144
allotments        5665
Name: count, dtype: int64

## Add coordinate system to predicted land cover

In [24]:
def get_new_lat_lon(old_lat, old_lon, dy, dx, r_earth = 6378.137):
    pi = math.pi
    new_latitude = old_lat + (dy / r_earth) * (180 / pi)
    new_longitude = old_lon + ((dx / r_earth) * (180 / pi) / math.cos(new_latitude * pi/180))
    return new_latitude, new_longitude

def get_new_lat(start_lat, start_lon, dx, pixels):
    new_coords = [(start_lat, start_lon)]
    for i in range(0, pixels-1):
        lat, lon = get_new_lat_lon(start_lat, start_lon, 0.0, dx)
        new_coords.append((lat, lon))
        start_lat = lat
        start_lon = lon
    return new_coords

def get_coords_for_pixels(center_lat, center_lon, size):
    start_lat, start_lon = get_new_lat_lon(center_lat, center_lon, 0.219, -0.212) 
    coords = []

    for i in range(0, size):
        coords_row = get_new_lat(start_lat, start_lon, 0.0004140625, size)
        start_lat, start_lon = get_new_lat_lon(start_lat, start_lon, -0.000412085938, 0.0)
        coords.append(coords_row)
    return coords

def large_buffer(coords_array, item, size):
    coords_pixels = []
    for i in range(0, 9):
        coord = get_coords_for_pixels(coords_array[item, 0, i, 2], coords_array[item, 0, i, 3], size)      
        coords_pixels.append(coord)
    coords_pixels = np.asarray(coords_pixels)
    row_1 = np.hstack((coords_pixels[0,:,:], coords_pixels[1,:,:], coords_pixels[2,:,:]))
    row_2 = np.hstack((coords_pixels[3,:,:], coords_pixels[4,:,:], coords_pixels[5,:,:]))
    row_3 = np.hstack((coords_pixels[6,:,:], coords_pixels[7,:,:], coords_pixels[8,:,:]))
    coords_pixels = np.vstack((row_1, row_2, row_3))
    return coords_pixels

def entropy(P, eps=1e-7):
    if len(P) == 0:
        entropy = np.NAN
    else:
        entropy = -(np.sum((P.values+eps) * np.log(P.values+eps)) / np.log(len(P.columns)))
    return entropy

In [23]:
shp21 = pd.read_pickle(dir_h + "shp21_preliminar.pkl")
shp21 = shp21[["idhous21", "g_lat", "g_lon", "npa"]].drop_duplicates().reset_index(drop=True)
image_order = np.load(dir_rs +"CH_images/image_4658_order.npy")
mask = np.isin(image_order, shp21.idhous21)
image_order = image_order[mask]

In [25]:
order_map = {}
for i in range(0,len(image_order)):
    dict_ = {image_order[i]: i}
    order_map.update(dict_)

In [26]:
shp21 = shp21.sort_values(by="idhous21", key=lambda x: x.map(order_map))

### buffer 420m

In [ ]:
#mask_center_0_2299 = np.load(dir_rs + "CH_images/mask_center_1d_0_2299.npy")
#mask_center_2300_4658 = np.load(dir_rs + "CH_images/mask_center_1d_2300_4658.npy")
#mask_center_all = np.concatenate([mask_center_0_2299, mask_center_2300_4658], axis=0)

mask_center_all = np.load(dir_rs + "CH_images/mask_center_all.npy")
mask_center_all.shape

Note:
* meadows can be private gardens or meadows cultivated by farmers
* forests can only consists of tree land cover
* recreation and playground can be from meadows and dev_space land cover
* for commercial, industrial, retail, warehouse, and sports_centre only buildings are considered

In [ ]:
cover_map = {1:"bareland", 
          2:"meadow", 
          3:"dev_space", 
          4:"roads",
          5:"trees", 
          6:"water",
          7:"agriculture",
          8:"buildings"} 

cols_green = ["trees_forest", "trees_park", "trees_other", "meadow_parks", "meadow_recreation",
                "meadow_garden", "meadow_playground", "meadow_other", "agri_allotments",
               "agri_other"]

cols_mixed_use = ["residential", "commercial", "groceries", "CCE", "recreation_use", "public_services"]

columns = ['idhous21', 'g_lat', 'g_lon', 'trees_forest', 'trees_park', 'trees_other', 'meadow_parks',
               'meadow_recreation', 'meadow_garden', 'meadow_playground', 'meadow_other', 'agri_allotments',
               'agri_other', 'residential', 'commercial', 'groceries', 'CCE', 'recreation_use', 'public_services',
               'commercial_groceries', 'entropy_6', 'entropy_5',]

data_land_use = pd.DataFrame([], columns=columns)
img_count_ten = 0
img_count = 0

for i in range(0, len(mask_center_all)):
    coords_mask = get_coords_for_pixels(shp21.iloc[i].g_lat, shp21.iloc[i].g_lon, 1024)
    coords_values = np.concatenate([coords_mask, np.expand_dims(mask_center_all[i], axis=2)], axis=2)
    gdf = gpd.GeoDataFrame(pd.DataFrame(list(coords_values[:,:,2].flatten()), columns=["land_cover"]), geometry=gpd.points_from_xy(list(coords_values[:,:,1].flatten()), list(coords_values[:,:,0].flatten())))
    gdf = gdf.set_crs("EPSG:4326")
    gdf = gdf.assign(land_cover = gdf.land_cover.map(cover_map))
    gdf["pixel_index"] = gdf.index
    usage = gpd.overlay(gdf, land_usage[land_usage.idhous21==shp21.iloc[i].idhous21], how="intersection")
    gdf = gdf.merge(usage[["pixel_index", "idhous21", "osmid", "osmid_count", "double_usage",
                       "amenity_weight", "use_green", "use_mixed"]], how="left", on="pixel_index")
    gdf["amenity_weight"] = np.where(gdf.amenity_weight.isna(), 1.0, gdf.amenity_weight)
    gdf["use_green"] = np.where(gdf.use_green.isna(), gdf.land_cover, gdf.use_green)
    mask = gdf[gdf.duplicated(subset="pixel_index", keep=False)].groupby("pixel_index").amenity_weight.transform("sum") > 1
    same_pixel = gdf[gdf.duplicated(subset="pixel_index", keep=False)][mask]
    same_pixel["amenity_weight"] = same_pixel.amenity_weight * 0.5
    gdf = gdf[~gdf.pixel_index.isin(same_pixel.pixel_index)]
    gdf = pd.concat([gdf, same_pixel])
    
    gdf["use_green"] = np.where((gdf.land_cover=="trees") & (gdf.use_green=="forest"), "trees_forest",
                           np.where((gdf.land_cover=="trees") & (gdf.use_green=="parks"), "trees_park",
                                   np.where((gdf.land_cover=="trees") & (~gdf.use_green.isin(["forest", "parks"])), "trees_other",
                                           np.where((gdf.land_cover=="meadow") & (gdf.use_green=="parks"), "meadow_parks",
                                                   np.where((gdf.land_cover=="meadow") & (gdf.use_green=="recreation"), "meadow_recreation",
                                                           np.where((gdf.land_cover=="meadow") & (gdf.use_green=="garden"), "meadow_garden",
                                                                   np.where((gdf.land_cover=="meadow") & (gdf.use_green=="playground"), "meadow_playground",
                                                                           np.where((gdf.land_cover=="meadow") & (~gdf.use_green.isin(["parks", "recreation", "garden", "playground"])), "meadow_other",
                                                                                   np.where((gdf.land_cover=="agriculture") & (gdf.use_green=="allotments"), "agri_allotments",
                                                                                           np.where((gdf.land_cover=="agriculture") & (gdf.use_green!="allotments"), "agri_other", gdf.use_green))))))))))

    values = []
    for col in cols_green:
            proportion = sum(np.where(gdf.use_green==col, 1, 0) * gdf.amenity_weight) / sum(gdf.amenity_weight)
            values.append(proportion)
    data_green_use = pd.DataFrame([values], columns=cols_green)

    values = []
    gdf_subset = gdf[(~gdf.use_mixed.isna()) & (~gdf.use_mixed.isin(["other", "industry"]))]
    
    for col in cols_mixed_use:
        if len(gdf_subset) == 0:
            proportion = np.NAN
        else:
            proportion = sum(np.where(gdf_subset.use_mixed==col, 1, 0) * gdf_subset.amenity_weight) / sum(gdf_subset.amenity_weight)
        values.append(proportion)
    data_mixed_use = pd.DataFrame([values], columns=cols_mixed_use)
    data_mixed_use["entropy_6"] = entropy(data_mixed_use)
    data_mixed_use["commercial_groceries"] = data_mixed_use.commercial + data_mixed_use.groceries
    data_mixed_use["entropy_5"] = entropy(data_mixed_use[["residential", "commercial_groceries", "CCE", "recreation_use", "public_services"]])
    additional_info = pd.DataFrame([[shp21.iloc[i].idhous21, shp21.iloc[i].g_lat, shp21.iloc[i].g_lon]], columns=["idhous21", "g_lat", "g_lon"])
    data_mixed_use = pd.concat([additional_info, data_green_use, data_mixed_use], axis=1)
    data_land_use = pd.concat([data_land_use, data_mixed_use], axis=0)

    img_count_ten += 1
    if img_count_ten == 10:
        img_count += 1
        img_count_ten=0
        print(img_count)

In [ ]:
data_land_use = data_land_use.reset_index(drop=True)
data_land_use

In [ ]:
#data_land_use.to_pickle(dir_h + "data_land_use_center.pkl")

### buffer 1260m

In [ ]:
#mask_stacked_0_599 = np.load(dir_rs + "CH_images/mask_stacked_0_599.npy")
#mask_stacked_600_1199 = np.load(dir_rs + "CH_images/mask_stacked_600_1199.npy")
#mask_stacked_1200_2399 = np.load(dir_rs + "CH_images/mask_stacked_1200_2399.npy")
#mask_stacked_2400_3599 = np.load(dir_rs + "CH_images/mask_stacked_2400_3599.npy")
#mask_stacked_3600_4658 = np.load(dir_rs + "CH_images/mask_stacked_3600_4658.npy")
#mask_stacked_all = np.concatenate([mask_stacked_0_599, mask_stacked_600_1199, mask_stacked_1200_2399, mask_stacked_2400_3599, mask_stacked_3600_4658], axis=0)

mask_stacked_all = np.load(dir_rs + "CH_images/mask_stacked_all.npy")

In [17]:
coords_array = np.load(dir_h + "coords_array.npy")

In [18]:
idx = []
for i in list(order_map.keys()):
    idx.append(list(np.unique(coords_array[:,0,:,0])).index(i))
    
coords_array = coords_array[idx]

In [ ]:
cover_map = {1:"bareland", 
          2:"meadow", 
          3:"dev_space", 
          4:"roads",
          5:"trees", 
          6:"water",
          7:"agriculture",
          8:"buildings"} 

cols_green = ["trees_forest", "trees_park", "trees_other", "meadow_parks", "meadow_recreation",
                "meadow_garden", "meadow_playground", "meadow_other", "agri_allotments",
               "agri_other"]

cols_mixed_use = ["residential", "commercial", "groceries", "CCE", "recreation_use", "public_services"]

columns = ['idhous21', 'g_lat', 'g_lon', 'trees_forest', 'trees_park', 'trees_other', 'meadow_parks',
               'meadow_recreation', 'meadow_garden', 'meadow_playground', 'meadow_other', 'agri_allotments',
               'agri_other', 'residential', 'commercial', 'groceries', 'CCE', 'recreation_use', 'public_services',
               'commercial_groceries', 'entropy_6', 'entropy_5',]

data_land_use = pd.DataFrame([], columns=columns)
img_count_ten = 0
img_count = 0


for i in range(0, len(mask_stacked_all)):
    coords_mask = large_buffer(coords_array, i, 1024)
    coords_values = np.concatenate([coords_mask, np.expand_dims(mask_stacked_all[i], axis=2)], axis=2)
    gdf = gpd.GeoDataFrame(pd.DataFrame(list(coords_values[:,:,2].flatten()), columns=["land_cover"]), geometry=gpd.points_from_xy(list(coords_values[:,:,1].flatten()), list(coords_values[:,:,0].flatten())))
    gdf = gdf.set_crs("EPSG:4326")
    gdf = gdf.assign(land_cover = gdf.land_cover.map(cover_map))
    gdf["pixel_index"] = gdf.index
    usage = gpd.overlay(gdf, land_usage[land_usage.idhous21==shp21.iloc[i].idhous21], how="intersection")
    gdf = gdf.merge(usage[["pixel_index", "idhous21", "osmid", "osmid_count", "double_usage",
                       "amenity_weight", "use_green", "use_mixed"]], how="left", on="pixel_index")
    gdf["amenity_weight"] = np.where(gdf.amenity_weight.isna(), 1.0, gdf.amenity_weight)
    gdf["use_green"] = np.where(gdf.use_green.isna(), gdf.land_cover, gdf.use_green)
    mask = gdf[gdf.duplicated(subset="pixel_index", keep=False)].groupby("pixel_index").amenity_weight.transform("sum") > 1
    same_pixel = gdf[gdf.duplicated(subset="pixel_index", keep=False)][mask]
    same_pixel["amenity_weight"] = same_pixel.amenity_weight * 0.5
    gdf = gdf[~gdf.pixel_index.isin(same_pixel.pixel_index)]
    gdf = pd.concat([gdf, same_pixel])
    
    gdf["use_green"] = np.where((gdf.land_cover=="trees") & (gdf.use_green=="forest"), "trees_forest",
                           np.where((gdf.land_cover=="trees") & (gdf.use_green=="parks"), "trees_park",
                                   np.where((gdf.land_cover=="trees") & (~gdf.use_green.isin(["forest", "parks"])), "trees_other",
                                           np.where((gdf.land_cover=="meadow") & (gdf.use_green=="parks"), "meadow_parks",
                                                   np.where((gdf.land_cover=="meadow") & (gdf.use_green=="recreation"), "meadow_recreation",
                                                           np.where((gdf.land_cover=="meadow") & (gdf.use_green=="garden"), "meadow_garden",
                                                                   np.where((gdf.land_cover=="meadow") & (gdf.use_green=="playground"), "meadow_playground",
                                                                           np.where((gdf.land_cover=="meadow") & (~gdf.use_green.isin(["parks", "recreation", "garden", "playground"])), "meadow_other",
                                                                                   np.where((gdf.land_cover=="agriculture") & (gdf.use_green=="allotments"), "agri_allotments",
                                                                                           np.where((gdf.land_cover=="agriculture") & (gdf.use_green!="allotments"), "agri_other", gdf.use_green))))))))))

    values = []
    for col in cols_green:
            proportion = sum(np.where(gdf.use_green==col, 1, 0) * gdf.amenity_weight) / sum(gdf.amenity_weight)
            values.append(proportion)
    data_green_use = pd.DataFrame([values], columns=cols_green)

    values = []
    gdf_subset = gdf[(~gdf.use_mixed.isna()) & (~gdf.use_mixed.isin(["other", "industry"]))]
    
    for col in cols_mixed_use:
        if len(gdf_subset) == 0:
            proportion = np.NAN
        else:
            proportion = sum(np.where(gdf_subset.use_mixed==col, 1, 0) * gdf_subset.amenity_weight) / sum(gdf_subset.amenity_weight)
        values.append(proportion)
    data_mixed_use = pd.DataFrame([values], columns=cols_mixed_use)
    data_mixed_use["entropy_6"] = entropy(data_mixed_use)
    data_mixed_use["commercial_groceries"] = data_mixed_use.commercial + data_mixed_use.groceries
    data_mixed_use["entropy_5"] = entropy(data_mixed_use[["residential", "commercial_groceries", "CCE", "recreation_use", "public_services"]])
    additional_info = pd.DataFrame([[shp21.iloc[i].idhous21, shp21.iloc[i].g_lat, shp21.iloc[i].g_lon]], columns=["idhous21", "g_lat", "g_lon"])
    data_mixed_use = pd.concat([additional_info, data_green_use, data_mixed_use], axis=1)
    data_land_use = pd.concat([data_land_use, data_mixed_use], axis=0)

    img_count_ten += 1
    if img_count_ten == 10:
        img_count += 1
        img_count_ten=0
        print(img_count)

In [ ]:
data_land_use = data_land_use.reset_index(drop=True)

In [37]:
#data_land_use.to_pickle(dir_h + "data_land_use_stacked.pkl")

## Combine land types with SHP data

In [44]:
shp21 = pd.read_pickle(dir_h + "shp21_preliminar.pkl")

In [45]:
data_land_use_630 = pd.read_pickle(dir_h + "data_land_use_stacked.pkl")
data_land_use_630 = data_land_use_630.drop(columns=["g_lat", "g_lon"]).add_suffix("_630")
data_land_use_630 = data_land_use_630.rename(columns={"idhous21_630": "idhous21"})

In [46]:
data_land_use_210 = pd.read_pickle(dir_h + "data_land_use_center.pkl")
data_land_use_210 = data_land_use_210.drop(columns=["g_lat", "g_lon"]).add_suffix("_210")
data_land_use_210 = data_land_use_210.rename(columns={"idhous21_210": "idhous21"})

In [ ]:
shp21 = shp21.merge(data_land_use_630, how="left", on="idhous21")
shp21 = shp21.merge(data_land_use_210, how="left", on="idhous21")

In [48]:
shp21.to_pickle(dir_h + "shp_stat_analysis.pkl")
shp21.to_csv(dir_h + "shp_stat_analysis.csv", index=False)

=> proceed with file ```analysis_land_types.R``` and ```visulaizations.ipynb```